<a href="https://colab.research.google.com/github/xinal88/CprogrammingIntroduction/blob/main/Santa_2024_The_Perplexity_Permutation_Puzzle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preprocessing the data

In [11]:
%%time
import pandas as pd
# Load the dataset
train_df = pd.read_csv("/content/sample_submission.csv")  # Example path
# test_df = pd.read_csv("test.csv")
# Display structure
print(train_df.head())

   id                                               text
0   0  advent chimney elf family fireplace gingerbrea...
1   1  advent chimney elf family fireplace gingerbrea...
2   2  yuletide decorations gifts cheer holiday carol...
3   3  yuletide decorations gifts cheer holiday carol...
4   4  hohoho candle poinsettia snowglobe peppermint ...
CPU times: user 13.2 ms, sys: 2.03 ms, total: 15.2 ms
Wall time: 20 ms


# Language Model for Perplexity

In [12]:

from transformers import GPT2LMHeadModel, GPT2Tokenizer

import numpy as np
# Load model and tokenizer
model_name = "gpt2"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

def calculate_perplexity(sequence):
  inputs = tokenizer(sequence, return_tensors="pt", truncation = True)
  # Converts the input text (sequence) into tokenized input IDs that GPT-2 can process.
  # return_tensors="pt": Ensures that the output is in PyTorch tensor format, which is required for the GPT-2 model.
  # truncation=True: Truncates text that exceeds the model's maximum input length. (typically 1024 tokens)

  outputs = model(**inputs, labels=inputs["input_ids"])
  # labels=inputs["input_ids"]: Used to compute the loss between the model's predictions and the input tokens (language modeling loss).

  loss = outputs.loss.item()
  # Retrieves the scalar value of the loss (cross-entropy).
  return np.exp(loss)
  # Perplexity is the exponential of the cross-entropy loss, representing how "confused" the model is about predicting the text.

#Example usage
sentence = "Santa loves coding puzzle durint Christmas."

perplexity = calculate_perplexity(sentence)
# Perplexity: a metric used in NLP to evaluate how well a language model predicts a sequence of text. A lower perplexity value indactes
# better predictive performance by the model for the given input
print(f"Perplexity of the sentence '{sentence}': {perplexity}")

Perplexity of the sentence 'Santa loves coding puzzle durint Christmas.': 16148.121124604595


# Dynamic Programming for Sentence Reconstruction

Use dynamic programming to build coherent sentences efficiently
- Define a state as the current position in the word sequence
- Define a transition cost as the perplexity of moving to the next word

In [13]:
import numpy as np

def dp_min_perplexity(words):
  n = len(words) # Number of words in the input list words
  dp = np.full((n, n), float('inf')) # A 2D DP table dp is initialized with dimensions (n, n) filled with inf (infinity).
  # This table will store the cumulative perplexities for different word sequences.
  dp[0][0] = 0 # Starting point
  for i in range(1, n):
    for j in range(i):
      sequence = " ".join(words[:i])
      dp[i][j] = calculate_perplexity(sequence) + dp[i-1][j]

# Traceback for best sequence
  best_path = []
  cur = np.argmin(dp[-1])
  for i in range(n-1, -1, -1):
    best_path.append(words[cur])
    cur = np.argmin(dp[i-1]) if i > 0 else 0
  return " ".join(best_path[::-1])

# Example usage
tokens = [ "Santa", "loves", "coding", "puzzles", "during", "Christmas"]
best_sequence = dp_min_perplexity(tokens)
print("Best Sequence:", best_sequence)


Best Sequence: Santa Santa Santa Santa Santa Santa


# Rearranaging Words

In [14]:
import itertools

def generate_permutation(tokens):
  return list(itertools.permutations(tokens))

# Generate and evaluate permutations
tokens = ["Santa", "loves", "coding", "puzzles"]
permutations = generate_permutation(tokens)

# Score each permutation
best_sequence = None
lowest_perplexity = float('inf')

for perm in permutations:
  candidate = " ".join(perm)
  perplexity = calculate_perplexity(candidate)
  if perplexity < lowest_perplexity:
    lowest_perplexity = perplexity
    best_sequence = candidate

print("Best Sequence:", best_sequence)
print("Lowest Perplexity:", lowest_perplexity)


Best Sequence: puzzles Santa loves coding
Lowest Perplexity: 3797.983879804625


# Beam Search (Scalable Optimization)


In [15]:
import heapq

def beam_search(tokens, beam_width=3):
  sequences = [{"", 1.0}] # Initialize beam with empty sequence
  for token in tokens:
    all_candidates = []
    for seq, score in sequences:
      new_seq = f"{seq} {token}".strip()
      new_score = score * calculate_perplexity(new_seq)
      all_candidates.append((new_seq, new_score))

      # Keep top `beam_width` candidates
    sequences = heapq.nsmallest(beam_width, all_candidates, key=lambda x: x[1])
  return sequences[0] # Best sequence

#Example usage
tokens = ["Santa", "loves", "coding", "puzzles"]
best_seq = beam_search(tokens)
print(f"Best sequence: {best_seq[0]} with perplexity {best_seq[1]}")

Best sequence: Santa loves coding puzzles with perplexity nan


In [16]:
#@title Dynamic programming for Sentence Reconstruction
# Use dynamic programming to build coherent sentences efficiently
#**Define a state as the current position in the word sequence
#**Define a transition cost as the perplexity of moving to the next word

%%time
import numpy as np
def dp_min_perplexity(words):
  n = len(words)
  dp = np.full((n, n), float('inf')) # DP Table
  dp[0][0] = 0
  for i in range(1, n):
    for j in range(i):
      sequence = " ".join(words[:i])
      dp[i][j] = calculate_perplexity(sequence) + dp[i-1][j]

  # Traceback for best sequence
  best_path = []
  curr = np.argmin(dp[-1])
  for i in range(n-1, -1, -1):
    best_path.append(words[curr])
    curr = np.argmin(dp[i-1]) if i > 0 else 0
  return " ".join(best_path[::-1])

# Example usage
tokens = ["Santa", "loves", "coding", "puzzles", "during", "Christmas"]
best_sequence = dp_min_perplexity(tokens)
print("Best Sequence:", best_sequence)

Best Sequence: Santa Santa Santa Santa Santa Santa
CPU times: user 1.39 s, sys: 395 µs, total: 1.39 s
Wall time: 1.4 s


In [17]:
%%time
predictions = []
for i, row in train_df.iterrows():
  scrambled_words = row["text"].split()
  best_sequence = dp_min_perplexity(scrambled_words)
  predictions.append(best_sequence)

CPU times: user 48min 7s, sys: 17.9 s, total: 48min 25s
Wall time: 49min 2s


In [ ]:
%%time
predictions = []
for i, row in train_df.iterrows():
    scrambled_words = row["text"].split()
    best_sequence = dp_min_perplexity(scrambled_words)
    predictions.append(best_sequence)

In [19]:
#@title Create the submission file
%%time
submission_df = pd.DataFrame({
    "id": train_df["id"],
    "predicted_text": predictions
})

# Save submission file
submission_df.to_csv("submission.csv", index=False)
print("Submission file created: submission.csv")

Submission file created: submission.csv
CPU times: user 3.62 ms, sys: 0 ns, total: 3.62 ms
Wall time: 3.91 ms
